In [21]:
import pandas as pd, numpy as np
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, roc_curve
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


In [22]:
train_df = pd.read_csv("data/twitter_data_train_multiclass.csv")
test_df = pd.read_csv("data/twitter_data_test_multiclass.csv")

print(train_df.shape)
print(test_df.shape)

(9446, 236)
(1667, 236)


In [23]:
to_drop = [
    'Unnamed: 0',
    'account_type', 
    'account_type_multi',
    'id', 
    'name', 
    'screen_name',
    'profile_image_url',
    'protected',
    'verified',
    'description',
    'description_processed',
    'tweets_list',
    'tweets_list_processed',
    'url',
    'profile_use_background_image', 
    'profile_background_tile'
    ]

scaler = MinMaxScaler() 
train_df[['statuses_count', 'favourites_count', 'followers_count', 'friends_count', 'number_of_mentions', 'listed_count', 'number_of_tags']] = scaler.fit_transform(train_df[['statuses_count', 'favourites_count', 'followers_count', 'friends_count', 'number_of_mentions', 'listed_count', 'number_of_tags']])
test_df[['statuses_count', 'favourites_count', 'followers_count', 'friends_count', 'number_of_mentions', 'listed_count', 'number_of_tags']] = scaler.transform(test_df[['statuses_count', 'favourites_count', 'followers_count', 'friends_count', 'number_of_mentions', 'listed_count', 'number_of_tags']])

X_train, y_train = train_df.drop(to_drop, axis=1), train_df['account_type_multi']
X_test, y_test = test_df.drop(to_drop, axis=1), test_df['account_type_multi']

# validation split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15)

In [24]:
from keras.utils import to_categorical
y_train_cat = to_categorical(y_train)
y_val_cat = to_categorical(y_val)
y_test_cat  = to_categorical(y_test)

In [25]:
model = Sequential()

model.add(Dense(100, activation='relu', input_dim=220))
model.add(Dropout(.15))
model.add(Dense(500, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(6, activation='softmax'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

model.fit(X_train, y_train_cat, epochs=20, validation_data=(X_val, y_val_cat))


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 100)               22100     
                                                                 
 dropout_5 (Dropout)         (None, 100)               0         
                                                                 
 dense_21 (Dense)            (None, 500)               50500     
                                                                 
 dense_22 (Dense)            (None, 500)               250500    
                                                                 
 dense_23 (Dense)            (None, 6)                 3006      
                                                                 
Total params: 326,106
Trainable params: 326,106
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
251/251 [============================

Train

In [28]:
import time

from sklearn.metrics import f1_score

best_clf = model

start_time = time.time()

y_pred_train = best_clf.predict(X_train)
y_pred_train = np.argmax(y_pred_train,axis=1)

time_taken = time.time() - start_time
print("Total time taken for the program execution", time_taken) # seconds
print(classification_report(y_train, y_pred_train, digits=5))
# roc_auc_score(y_test, y_pred_test)
print("F1 weighted: ", f1_score(y_train, y_pred_train, average='weighted'))

251/251 [==============================] - 0s 916us/step
Total time taken for the program execution 0.3290395736694336
              precision    recall  f1-score   support

           0    0.97891   0.99501   0.98690      2006
           1    0.99609   0.99177   0.99392      2309
           2    1.00000   0.96538   0.98239       260
           3    0.99775   0.99455   0.99615      3119
           4    0.98513   0.97426   0.97967       272
           5    1.00000   0.98413   0.99200        63

    accuracy                        0.99215      8029
   macro avg    0.99298   0.98419   0.98850      8029
weighted avg    0.99223   0.99215   0.99216      8029

F1 weighted:  0.9921600183867991


test

In [29]:
start_time = time.time()

y_pred_test = best_clf.predict(X_test)
y_pred_test = np.argmax(y_pred_test,axis=1)

time_taken = time.time() - start_time
print("Total time taken for the program execution", time_taken) # seconds
print(classification_report(y_test, y_pred_test, digits=5))
# roc_auc_score(y_test, y_pred_test)
print("F1 weighted: ", f1_score(y_test, y_pred_test, average='weighted'))

53/53 [==============================] - 0s 869us/step
Total time taken for the program execution 0.11520028114318848
              precision    recall  f1-score   support

           0    0.95508   0.97821   0.96651       413
           1    0.99130   0.99130   0.99130       460
           2    1.00000   0.89655   0.94545        58
           3    0.99544   0.98348   0.98943       666
           4    0.89655   0.98113   0.93694        53
           5    1.00000   0.94118   0.96970        17

    accuracy                        0.98080      1667
   macro avg    0.97306   0.96198   0.96655      1667
weighted avg    0.98136   0.98080   0.98087      1667

F1 weighted:  0.9808662494428282
